In [146]:
import pandas as pd
import numpy as np

import pickle
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import CountFrequencyEncoder

from sklearn.preprocessing import StandardScaler


In [147]:
dataset = pd.read_csv("../data/raw/loan_sanction_train.csv")
dataset.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [148]:
# configuracipon del Pipeline
TARGET = "Loan_Status"
VARS_TO_DROP = ["Loan_ID",TARGET]
CONTINUE_VARS_TO_IMPUTATION= ["LoanAmount","Loan_Amount_Term","Credit_History","ApplicantIncome","CoapplicantIncome"]
CATEGORICAL_VARS_TO_IMPUTATION=["Gender","Married","Dependents","Education","Self_Employed","Property_Area"]
OHE_VAR_TO_ENCODE = ["Gender","Married","Education","Self_Employed"]
FREQENC_VAR_TO_ENCODE = ["Dependents","Property_Area"]



In [149]:
x_features = dataset.drop(labels=VARS_TO_DROP,axis=1)
y_target = dataset[TARGET]

x_train, x_test, y_train, y_test = train_test_split(x_features,y_target, test_size=0.3,shuffle=True,random_state=2025)

In [150]:
x_train.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area'],
      dtype='object')

In [151]:
from sklearn.pipeline import Pipeline

loan_prediction_model = Pipeline([
    # imputacion de variables continuas
    ("continues_var_mean_imputacion",MeanMedianImputer(imputation_method="mean",variables=CONTINUE_VARS_TO_IMPUTATION)),
    #imputacion de variables categoricas
    ("categorical_var_freq_imputation",CategoricalImputer(imputation_method="frequent",variables=CATEGORICAL_VARS_TO_IMPUTATION)),
    #codificaciones de las variables categoricas
    ("categorical_encoding_ohe",OneHotEncoder(variables=OHE_VAR_TO_ENCODE,drop_last=True)),
    ("categorical_encoding_freq_enc",CountFrequencyEncoder(encoding_method="count",variables=FREQENC_VAR_TO_ENCODE)),
    # estandarizacion de variables
    ("feature_scaling",StandardScaler())
])

In [152]:
loan_prediction_model.fit(x_train)

Pipeline(steps=[('continues_var_mean_imputacion',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['LoanAmount', 'Loan_Amount_Term',
                                              'Credit_History',
                                              'ApplicantIncome',
                                              'CoapplicantIncome'])),
                ('categorical_var_freq_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=['Gender', 'Married',
                                               'Dependents', 'Education',
                                               'Self_Employed',
                                               'Property_Area'])),
                ('categorical_encoding_ohe',
                 OneHotEncoder(drop_last=True,
                               variables=['Gender', 'Married', 'Education',
                                          'Self_Employed'])),
                ('categorical_encoding_freq_enc',
                 CountFrequencyEncoder(variables=['Dependents',
                                                  'Property_Area'])),
                ('feature_scaling', StandardScaler())])

In [153]:
x_features_processed = loan_prediction_model.transform(x_train)
df_features_process = pd.DataFrame(x_features_processed,columns=x_train.columns)
df_features_process[TARGET] = y_train.reset_index()[TARGET]

# guardamos los datos para entrenar los modelos.
df_features_process.to_csv('../data/processed/features_for_model.csv', index=False)
df_features_process.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0.837587,-0.447486,-0.512235,-1.106041,0.269448,4.565051e-01,1.122887,0.489780,1.370837,0.551318,0.371391,Y
1,0.837587,0.045112,0.999805,1.325673,0.269448,4.565051e-01,1.122887,0.489780,-0.729481,0.551318,0.371391,N
2,0.837587,-0.446417,-0.512235,-1.094062,0.269448,4.565051e-01,-1.341146,-2.041733,1.370837,0.551318,0.371391,Y
3,-1.161664,0.341495,-0.512235,0.451214,0.269448,-3.161607e-16,-1.341146,0.489780,-0.729481,-1.813836,0.371391,Y
4,0.837587,-0.432216,0.116602,-0.375329,0.269448,-2.391217e+00,1.122887,0.489780,-0.729481,-1.813836,0.371391,N


In [154]:
x_test[TARGET] = y_test
x_test.to_csv('../data/processed/test_dataset.csv', index=False)

with open('../artifacts/pipeline.pkl', 'wb') as f:
    pickle.dump(loan_prediction_model, f)